Alvienas Yandika
alvienasyandika@gmail.com
Kabupaten Bekasi

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alvienasyandika","key":"1fc7a59a1886877f717274b7fb39b742"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.jspm

chmod: cannot access '/root/.kaggle/kaggle.jspm': No such file or directory


In [ ]:
! kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           9066        270  1.0              
mattop/alcohol-consumption-per-capita-2016                      Alcohol Consumption Per Capita 2016                   4KB  2022-12-09 00:03:11            885         33  1.0              
thedevastator/jobs-dataset-from-glassdoor                       Salary Prediction                                     3MB  2022-11-16 13:52:31           5819        128  1.0              
swaptr/fifa-world-cup-2022-match-data                       

In [ ]:
! kaggle datasets download -d danofer/sarcasm

 98% 211M/216M [00:01<00:00, 197MB/s]
100% 216M/216M [00:01<00:00, 176MB/s]


In [ ]:
!unzip sarcasm.zip

Archive:  sarcasm.zip
  inflating: test-balanced.csv       
  inflating: test-unbalanced.csv     
  inflating: train-balanced-sarc.csv.gz  
  inflating: train-balanced-sarcasm.csv  


In [ ]:
import pandas as pd
df = pd.read_csv('train-balanced-sarcasm.csv')
# df.SentimentText=df.SentimentText.astype(str)
df = df.dropna()
# dataset = pd.DataFrame(data=df)
# dataset.astype('int32').dtypes

In [ ]:
len(df)
# df.info()

1010773

In [ ]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [ ]:
# category = pd.get_dummies(df.downs)
# df_baru = pd.concat([df, category], axis=1)
# df_baru = df_baru.drop(columns='downs')
# df_baru
desired_dataset = ['label', 'comment',	'author',	'subreddit',	'score',	'ups',	'downs',	'date',	'created_utc',	'parent_comment']
for i in df.columns:
  if i not in desired_dataset:
    df = df.drop(columns=[i])

df = df.loc[df['subreddit'] == 'BlackPeopleTwitter']

df = df.drop(columns=['subreddit'])

In [ ]:
df.head()

,label,comment,author,score,ups,downs,date,created_utc,parent_comment
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
393,0,He's really riding that pussy.,CoastalSailing,55,-1,-1,2016-11,2016-11-12 22:07:27,omw
401,0,I like when the juice just dribbles down your ...,jman12234,30,-1,-1,2016-11,2016-11-13 06:09:51,I remember doing this with fruit rollups' and ...
420,0,Underrated comment,DownvoteDaemon,1,-1,-1,2016-11,2016-11-05 01:54:25,"Dog smoked a blunt dipped in codeine, the cat ..."
421,0,Them hips don't lie.,DownvoteDaemon,114,-1,-1,2016-12,2016-12-16 06:40:22,I think the way the hips are shaped it makes i...


In [ ]:
len(df)

1831

In [ ]:
komen = df['comment'].values
down = df[['label','downs']].values

In [ ]:
from sklearn.model_selection import train_test_split
komen_train, komen_test, down_train, down_test = train_test_split(komen, down, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words= 15, oov_token='<oov>')
tokenizer.fit_on_texts((komen_train)) 
tokenizer.fit_on_texts(komen_test)
 
sekuens_train = tokenizer.texts_to_sequences(komen_train)
sekuens_test = tokenizer.texts_to_sequences(komen_test)

In [ ]:
padded_latih = pad_sequences(sekuens_train) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile (loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 100
history = model.fit(padded_latih, 
                    down_train, 
                    steps_per_epoch=30,
                    epochs=num_epochs, 
                    validation_data=(padded_test, down_test), verbose=2)

Epoch 1/100
30/30 - 2s - loss: -4.1925e-01 - accuracy: 0.9727 - val_loss: -1.2708e+00 - val_accuracy: 1.0000 - 2s/epoch - 75ms/step
Epoch 2/100
30/30 - 0s - loss: -1.1918e+00 - accuracy: 1.0000 - val_loss: -1.6653e+00 - val_accuracy: 1.0000 - 246ms/epoch - 8ms/step
Epoch 3/100
30/30 - 0s - loss: -2.2405e+00 - accuracy: 1.0000 - val_loss: -3.3310e+00 - val_accuracy: 1.0000 - 228ms/epoch - 8ms/step
Epoch 4/100
30/30 - 0s - loss: -3.7068e+00 - accuracy: 1.0000 - val_loss: -5.3006e+00 - val_accuracy: 1.0000 - 220ms/epoch - 7ms/step
Epoch 5/100
30/30 - 0s - loss: -5.5747e+00 - accuracy: 1.0000 - val_loss: -7.5375e+00 - val_accuracy: 1.0000 - 207ms/epoch - 7ms/step
Epoch 6/100
30/30 - 0s - loss: -7.1165e+00 - accuracy: 1.0000 - val_loss: -8.7104e+00 - val_accuracy: 1.0000 - 218ms/epoch - 7ms/step
Epoch 7/100
30/30 - 0s - loss: -8.8689e+00 - accuracy: 1.0000 - val_loss: -8.7200e+00 - val_accuracy: 1.0000 - 220ms/epoch - 7ms/step
Epoch 8/100
30/30 - 0s - loss: -5.3039e+00 - accuracy: 1.0000 - 